In [ ]:
import os
import re
import json
import multiprocessing as mp

import numpy as np
import pandas as pd

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.vasp.inputs import Potcar, Incar
from pymatgen.io.vasp.outputs import Outcar, Vasprun

ModuleNotFoundError: No module named 'functions'

In [ ]:
# Notebook outputs
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..')) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'outputs')
STRUCT_EXTXYZ_DIR = os.path.join(OUTPUT_DIR, 'structures_extxyz')
STRUCT_VASP_DIR = os.path.join(OUTPUT_DIR, 'structures_vasp')

os.makedirs(STRUCT_EXTXYZ_DIR, exist_ok=True)
os.makedirs(STRUCT_VASP_DIR, exist_ok=True)

print('PROJECT_ROOT:', PROJECT_ROOT)
print('OUTPUT_DIR:', OUTPUT_DIR)

In [ ]:
# Parse structure and VASP output files from SegregationProfile directory (parallel)
SegFolder = "/mnt/e/OneDrive/TiGB-Project/Data/SegregationProfile"

# Choose parallel backend: 'process' (true parallel) or 'thread' (I/O overlap)
PARALLEL_BACKEND = 'process'


def immediate_subdirs(path: str):
    """Immediate subdirectories (stdlib only), excluding notebook checkpoint dirs."""
    out = []
    for entry in os.scandir(path):
        if entry.is_dir() and entry.name != '.ipynb_checkpoints':
            out.append(entry.path)
    return out


# Build task list first (GB_type, element, system_folder)
tasks = []
for GB_folder in immediate_subdirs(SegFolder):
    GB_name = os.path.basename(GB_folder)
    for element_folder in immediate_subdirs(GB_folder):
        element_name = os.path.basename(element_folder)
        for system_folder in immediate_subdirs(element_folder):
            tasks.append((GB_name, element_name, system_folder))

print(f"Queued {len(tasks)} systems for parsing")

if PARALLEL_BACKEND == 'process':
    # CPU-bound parsing → processes. Keep it near CPU count to avoid thrashing on big XML/OUTCARs.
    max_workers = max(1, (os.cpu_count() or 8) - 1)
else:
    # I/O-heavy parsing → threads.
    max_workers = min(32, (os.cpu_count() or 8) * 2)

print(f"Backend={PARALLEL_BACKEND} max_workers={max_workers}")


def parse_system(GB_name, element_name, system_folder):
    system_name = os.path.basename(system_folder)
    job_name = f"{GB_name}-{element_name}-{system_name}"

    # Extract distance from system name (e.g., "Al-25-d-0.00" -> 0.00)
    distance_match = re.search(r'd-([\d.]+)', system_name)
    distance = float(distance_match.group(1)) if distance_match else None

    row_data = {
        'job_name': job_name,
        'GB_type': GB_name,
        'element': element_name,
        'system': system_name,
        'distance': distance,
        'path': system_folder,
    }

    parse_errors = []

    # --- Structure (prefer CONTCAR, then {system}.vasp, then POSCAR) ---
    structure = None
    structure_file = None
    for struct_file in ['CONTCAR', f'{system_name}.vasp', 'POSCAR']:
        struct_path = os.path.join(system_folder, struct_file)
        if os.path.exists(struct_path):
            try:
                structure = Structure.from_file(struct_path)
                structure_file = struct_file
                break
            except Exception as e:
                parse_errors.append(f"structure:{struct_file}:{e}")

    if structure is not None:
        row_data['structure_file'] = structure_file
        row_data['num_atoms'] = len(structure)
        row_data['formula'] = structure.formula
        row_data['volume'] = structure.volume
        row_data['density'] = structure.density
        row_data['lattice_a'] = structure.lattice.a
        row_data['lattice_b'] = structure.lattice.b
        row_data['lattice_c'] = structure.lattice.c
        row_data['lattice_alpha'] = structure.lattice.alpha
        row_data['lattice_beta'] = structure.lattice.beta
        row_data['lattice_gamma'] = structure.lattice.gamma

        # Pymatgen Structure JSON
        try:
            s_dict = structure.as_dict()
            row_data['pmg_structure_json'] = json.dumps(s_dict, default=str)
        except Exception as e:
            parse_errors.append(f"pmg_structure_json:{e}")

        # ASE Atoms JSON (via pymatgen adaptor; returns MSONAtoms)
        try:
            a = AseAtomsAdaptor.get_atoms(structure)
            row_data['ase_atoms_json'] = json.dumps(a.as_dict(), default=str)
        except Exception as e:
            parse_errors.append(f"ase_atoms_json:{e}")

    # --- Prefer VASPRUN for energy/forces/stress ---
    row_data['energy_eV'] = None
    row_data['forces_eV_A'] = None
    row_data['stress_kbar_3x3'] = None

    vasprun_path = os.path.join(system_folder, 'vasprun.xml')
    if os.path.exists(vasprun_path):
        try:
            vr = Vasprun(vasprun_path, parse_dos=False, parse_eigen=False)
            row_data['energy_eV'] = float(vr.final_energy)

            try:
                last = vr.ionic_steps[-1]
                # forces: (N,3) eV/Å
                row_data['forces_eV_A'] = np.array(last.get('forces')).tolist() if last.get('forces') is not None else None
                # stress: (3,3) kBar
                row_data['stress_kbar_3x3'] = np.array(last.get('stress')).tolist() if last.get('stress') is not None else None
            except Exception as e:
                parse_errors.append(f"vasprun:ionic_steps:{e}")

            # Use final structure from vasprun if we didn't parse one earlier
            if structure is None:
                try:
                    structure = vr.final_structure
                    row_data['structure_file'] = 'vasprun.xml'
                    row_data['num_atoms'] = len(structure)
                    row_data['formula'] = structure.formula
                    row_data['volume'] = structure.volume
                    row_data['density'] = structure.density
                    row_data['lattice_a'] = structure.lattice.a
                    row_data['lattice_b'] = structure.lattice.b
                    row_data['lattice_c'] = structure.lattice.c
                    row_data['lattice_alpha'] = structure.lattice.alpha
                    row_data['lattice_beta'] = structure.lattice.beta
                    row_data['lattice_gamma'] = structure.lattice.gamma

                    s_dict = structure.as_dict()
                    row_data['pmg_structure_json'] = json.dumps(s_dict, default=str)

                    a = AseAtomsAdaptor.get_atoms(structure)
                    row_data['ase_atoms_json'] = json.dumps(a.as_dict(), default=str)
                except Exception as e:
                    parse_errors.append(f"vasprun:final_structure:{e}")

        except Exception as e:
            parse_errors.append(f"vasprun:{e}")

    # --- OUTCAR fallback energy-only ---
    outcar_path = os.path.join(system_folder, 'OUTCAR')
    if row_data['energy_eV'] is None and os.path.exists(outcar_path):
        try:
            outcar = Outcar(outcar_path)
            if hasattr(outcar, 'final_energy') and outcar.final_energy is not None:
                row_data['energy_eV'] = float(outcar.final_energy)
        except Exception as e:
            parse_errors.append(f"outcar:{e}")

    if row_data['energy_eV'] is not None and structure is not None and len(structure) > 0:
        row_data['energy_per_atom_eV'] = row_data['energy_eV'] / len(structure)

    # --- INCAR (pymatgen parser) ---
    row_data['incar'] = None
    incar_path = os.path.join(system_folder, 'INCAR')
    if os.path.exists(incar_path):
        try:
            incar = Incar.from_file(incar_path)
            incar_dict = dict(incar)
            row_data['incar'] = json.dumps(incar_dict, default=str)

            for k in ['SYSTEM', 'ENCUT', 'EDIFF', 'EDIFFG', 'ISPIN', 'ISMEAR', 'SIGMA', 'NSW', 'IBRION', 'ISIF', 'PREC', 'LREAL', 'LWAVE', 'LCHARG', 'GGA']:
                if k in incar_dict:
                    row_data[f'incar_{k.lower()}'] = incar_dict[k]
        except Exception as e:
            parse_errors.append(f"incar:{e}")

    # --- POTCAR valence electrons (pymatgen parser) ---
    row_data['potcar_symbols'] = None
    row_data['potcar_nelectrons_map'] = None
    row_data['potcar_total_nelectrons'] = None

    potcar_path = os.path.join(system_folder, 'POTCAR')
    if os.path.exists(potcar_path) and structure is not None:
        try:
            potcar = Potcar.from_file(potcar_path)

            nelectrons_map = {}
            potcar_symbols = []
            for ps in potcar:
                potcar_symbols.append(getattr(ps, 'symbol', None))
                el = getattr(ps, 'element', None)
                ne = getattr(ps, 'nelectrons', None)
                if el is not None and ne is not None and str(el) not in nelectrons_map:
                    nelectrons_map[str(el)] = float(ne)

            total_ne = 0.0
            for el, amt in structure.composition.get_el_amt_dict().items():
                if el in nelectrons_map:
                    total_ne += float(amt) * float(nelectrons_map[el])

            row_data['potcar_symbols'] = json.dumps(potcar_symbols, default=str)
            row_data['potcar_nelectrons_map'] = json.dumps(nelectrons_map, default=str)
            row_data['potcar_total_nelectrons'] = total_ne

        except Exception as e:
            parse_errors.append(f"potcar:{e}")

    if parse_errors:
        row_data['parse_errors'] = ' | '.join(parse_errors)

    return row_data


if PARALLEL_BACKEND == 'process':
    # Ensure fork on linux to avoid pickling issues with notebooks
    ctx = mp.get_context('fork')
    Executor = lambda **kw: ProcessPoolExecutor(mp_context=ctx, **kw)
else:
    Executor = ThreadPoolExecutor


data_rows = []
with Executor(max_workers=max_workers) as ex:
    futures = [ex.submit(parse_system, *t) for t in tasks]
    for i, fut in enumerate(as_completed(futures), start=1):
        data_rows.append(fut.result())
        if i % 50 == 0 or i == len(futures):
            print(f"Parsed {i}/{len(futures)}")

# Create dataframe from parsed data and sort by element
df = pd.DataFrame(data_rows)

df = df.sort_values(['element', 'GB_type', 'distance', 'system'], na_position='last').reset_index(drop=True)

print(f"\nTotal systems parsed: {len(df)}")
print(f"\nDataFrame columns: {df.columns.tolist()}")
df.head()

Queued 530 systems for parsing
Backend=process max_workers=11
Parsed 50/530
Parsed 100/530
Parsed 150/530
Parsed 200/530
Parsed 250/530
Parsed 300/530
Parsed 350/530
Parsed 400/530
Parsed 450/530
Parsed 500/530
Parsed 530/530

Total systems parsed: 530

DataFrame columns: ['GB_type', 'element', 'system', 'distance', 'path', 'structure_file', 'num_atoms', 'formula', 'volume', 'density', 'lattice_a', 'lattice_b', 'lattice_c', 'lattice_alpha', 'lattice_beta', 'lattice_gamma', 'final_energy', 'final_energy_per_atom', 'fermi_energy', 'magnetization', 'n_ionic_steps', 'incar', 'incar_system', 'incar_encut', 'incar_ediff', 'incar_ediffg', 'incar_ispin', 'incar_ismear', 'incar_sigma', 'incar_nsw', 'incar_ibrion', 'incar_isif', 'incar_prec', 'incar_lreal', 'incar_gga', 'potcar_symbols', 'potcar_nelectrons_map', 'potcar_total_nelectrons']


,GB_type,element,system,distance,path,structure_file,num_atoms,formula,volume,density,...,incar_sigma,incar_nsw,incar_ibrion,incar_isif,incar_prec,incar_lreal,incar_gga,potcar_symbols,potcar_nelectrons_map,potcar_total_nelectrons
0,C1,Al,Al-25-d-0.00,0.00,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,0.2,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0
1,C1,Al,Al-26-d-0.00,0.00,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,0.2,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0
2,C1,Al,Al-27-d-1.38,1.38,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,0.2,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0
3,C1,Al,Al-28-d-1.38,1.38,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,0.2,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0
4,C1,Al,Al-29-d-2.59,2.59,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,0.2,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0


In [ ]:
# Display summary statistics
print("Summary of parsed data:")
print(f"Number of GB types: {df['GB_type'].nunique()}")
print(f"GB types: {df['GB_type'].unique()}")
print(f"\nNumber of elements: {df['element'].nunique()}")
print(f"Elements: {df['element'].unique()}")

# Energy/forces/stress now come from vasprun.xml when available
if 'energy_eV' in df.columns:
    print(f"\nNumber of systems with energy data: {df['energy_eV'].notna().sum()}")
else:
    print("\nNo energy_eV column found")

print(f"Number of systems with structure data: {df['num_atoms'].notna().sum()}")

if 'incar' in df.columns:
    print(f"Number of systems with INCAR parsed: {df['incar'].notna().sum()}")
if 'potcar_total_nelectrons' in df.columns:
    print(f"Number of systems with POTCAR valence electrons parsed: {df['potcar_total_nelectrons'].notna().sum()}")
if 'forces_eV_A' in df.columns:
    print(f"Number of systems with forces parsed: {df['forces_eV_A'].notna().sum()}")
if 'stress_kbar_3x3' in df.columns:
    print(f"Number of systems with stress parsed: {df['stress_kbar_3x3'].notna().sum()}")

# Show basic statistics
if 'energy_eV' in df.columns:
    print(f"\nEnergy statistics (eV):")
    print(df['energy_eV'].describe())

Summary of parsed data:
Number of GB types: 6
GB types: ['C1' 'CSL13' 'CSL7' 'T1g' 'T1m' 'T2']

Number of elements: 5
Elements: ['Al' 'Cu' 'V' 'Zn' 'Zr']

Number of systems with energy data: 530
Number of systems with structure data: 530
Number of systems with INCAR parsed: 530
Number of systems with POTCAR valence electrons parsed: 530

Energy statistics (eV):
count    530.000000
mean    -515.389830
std      122.936797
min     -775.583314
25%     -495.880169
50%     -491.939772
75%     -402.949814
max     -393.837441
Name: final_energy, dtype: float64


In [ ]:
# Calculate segregation energy (E_seg)
# E_seg = E - E_zero, where E_zero is the average energy for systems far from GB (distance > 5)
energy_col = 'energy_eV' if 'energy_eV' in df.columns else None

if energy_col is None:
    raise ValueError('Expected energy_eV in df. Re-run Cell 2 to rebuild df.')

df_with_energy = df[df[energy_col].notna()].copy()

df_with_energy['E_seg'] = np.nan

for GB_type in df_with_energy['GB_type'].unique():
    for element in df_with_energy['element'].unique():
        mask = (df_with_energy['GB_type'] == GB_type) & (df_with_energy['element'] == element)
        if mask.sum() == 0:
            continue

        # Calculate zero energy as average of systems with distance > 5
        zero_mask = mask & (df_with_energy['distance'] > 5)
        if zero_mask.sum() > 0:
            zero_energy = df_with_energy.loc[zero_mask, energy_col].mean()
            df_with_energy.loc[mask, 'E_seg'] = df_with_energy.loc[mask, energy_col] - zero_energy

df_with_energy.head()

,GB_type,element,system,distance,path,structure_file,num_atoms,formula,volume,density,...,incar_nsw,incar_ibrion,incar_isif,incar_prec,incar_lreal,incar_gga,potcar_symbols,potcar_nelectrons_map,potcar_total_nelectrons,E_seg
0,C1,Al,Al-25-d-0.00,0.00,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0,0.149211
1,C1,Al,Al-26-d-0.00,0.00,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0,0.149529
2,C1,Al,Al-27-d-1.38,1.38,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0,0.011232
3,C1,Al,Al-28-d-1.38,1.38,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0,0.052966
4,C1,Al,Al-29-d-2.59,2.59,/mnt/e/OneDrive/TiGB-Project/Data/SegregationP...,CONTCAR,52,Ti51 Al1,915.14781,4.478555,...,60,2,2,Accurate,Auto,Pe,"[""Ti"", ""Al"", ""Ti""]","{""Ti"": 4.0, ""Al"": 3.0}",207.0,-0.111880


In [ ]:
# Write the main dataframe to a pickle (includes JSON blobs for structures, forces, stress)
pickle_path = os.path.join(OUTPUT_DIR, 'segregation_profile.pkl')
df.to_pickle(pickle_path)
print('Wrote:', pickle_path)

# Optional: also write a "flat" CSV that excludes large JSON/array columns
flat_cols = [c for c in df.columns if c not in ['pmg_structure_json', 'ase_atoms_json', 'forces_eV_A', 'stress_kbar_3x3']]
df[flat_cols].to_csv(os.path.join(OUTPUT_DIR, 'segregation_profile_flat.csv'), index=False)
print('Wrote flat CSV:', os.path.join(OUTPUT_DIR, 'segregation_profile_flat.csv'))

# --- Write two additional "minimal" dataframes ---
# 1) ASE-only dataframe (actual ASE atoms objects)
ase_records = []
for _, row in df[['job_name', 'pmg_structure_json']].dropna().iterrows():
    try:
        pmg_struct = Structure.from_dict(json.loads(row['pmg_structure_json']))
        atoms = AseAtomsAdaptor.get_atoms(pmg_struct)
        ase_records.append({'job_name': row['job_name'], 'ase_atoms': atoms})
    except Exception:
        continue

df_ase_only = pd.DataFrame(ase_records)
ase_only_path = os.path.join(OUTPUT_DIR, 'segregation_profile_ase_only.pkl')
df_ase_only.to_pickle(ase_only_path)
print('Wrote ASE-only df:', ase_only_path)

# 2) Pymatgen-JSON-only dataframe
pmg_only_cols = [c for c in ['job_name', 'pmg_structure_json'] if c in df.columns]
df_pmg_json_only = df[pmg_only_cols].copy()
pmg_only_path = os.path.join(OUTPUT_DIR, 'segregation_profile_pmg_json_only.pkl')
df_pmg_json_only.to_pickle(pmg_only_path)
print('Wrote pmg-json-only df:', pmg_only_path)

Index(['GB_type', 'element', 'system', 'distance', 'path', 'structure_file',
       'num_atoms', 'formula', 'volume', 'density', 'lattice_a', 'lattice_b',
       'lattice_c', 'lattice_alpha', 'lattice_beta', 'lattice_gamma',
       'final_energy', 'final_energy_per_atom', 'fermi_energy',
       'magnetization', 'n_ionic_steps', 'incar', 'incar_system',
       'incar_encut', 'incar_ediff', 'incar_ediffg', 'incar_ispin',
       'incar_ismear', 'incar_sigma', 'incar_nsw', 'incar_ibrion',
       'incar_isif', 'incar_prec', 'incar_lreal', 'incar_gga',
       'potcar_symbols', 'potcar_nelectrons_map', 'potcar_total_nelectrons',
       'E_seg'],
      dtype='object')

In [ ]:
# Write per-job structure files
# - extxyz: includes energy (eV), forces (eV/Å), stress (kBar 3x3 matrix) in file metadata
# - vasp: POSCAR-style structure file (cannot store forces/stress in POSCAR)

from ase.io import write as ase_write

n_written_xyz = 0
n_written_vasp = 0

for _, row in df.iterrows():
    job_name = row.get('job_name')
    if not job_name:
        continue

    pmg_json = row.get('pmg_structure_json')
    if not pmg_json:
        continue

    try:
        pmg_struct = Structure.from_dict(json.loads(pmg_json))
        atoms = AseAtomsAdaptor.get_atoms(pmg_struct)
    except Exception:
        continue

    # Attach metadata
    atoms.info['job_name'] = job_name
    atoms.info['GB_type'] = row.get('GB_type')
    atoms.info['element'] = row.get('element')
    atoms.info['system'] = row.get('system')
    if row.get('distance') is not None:
        atoms.info['distance'] = float(row['distance'])

    # Energy (eV)
    if row.get('energy_eV') is not None:
        atoms.info['energy'] = float(row['energy_eV'])

    # Forces: Nx3 (eV/Å)
    if row.get('forces_eV_A') is not None:
        try:
            atoms.arrays['forces'] = np.array(row['forces_eV_A'], dtype=float)
        except Exception:
            pass

    # Stress: store raw 3x3 in kBar (from VASP/vasprun)
    if row.get('stress_kbar_3x3') is not None:
        atoms.info['stress_kbar_3x3'] = row['stress_kbar_3x3']

    # Write extxyz
    xyz_path = os.path.join(STRUCT_EXTXYZ_DIR, f"{job_name}.extxyz")
    ase_write(xyz_path, atoms, format='extxyz')
    n_written_xyz += 1

    # Write VASP structure
    vasp_path = os.path.join(STRUCT_VASP_DIR, f"{job_name}.vasp")
    pmg_struct.to(fmt='poscar', filename=vasp_path)
    n_written_vasp += 1

print(f"Wrote extxyz: {n_written_xyz} -> {STRUCT_EXTXYZ_DIR}")
print(f"Wrote vasp:   {n_written_vasp} -> {STRUCT_VASP_DIR}")

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
# Preview
(df[['job_name','GB_type','element','system','distance','energy_eV']].head(10) if 'job_name' in df.columns else df.head(10))